In [1]:
import joblib
import pandas as pd
import anndata
import xarray as xr
from ALLCools.mcds.MCDS import MCDS
import seaborn as sns
import matplotlib.pyplot as plt

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/merge.py:18: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1829: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  'DataArray', pd.Series, pd.DataFrame, pd.Panel]:


## Load model

In [2]:
snmc_meta = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
clf = joblib.load(
    '/home/hanliu/project/mouse_rostral_brain/study/FinalPredictionModelRecipe/NonNeuronPrediction/Model/FinalClassifier.lib'
)

In [4]:
features = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/FinalPredictionModelRecipe/NonNeuronPrediction/Model/ModelFeature.csv',
    index_col=0
)
features.head()

,chrom,start,end
index,,,
15119,chr10,116200000,116300000
18319,chr13,63200000,63300000
22270,chr17,10400000,10500000
8427,chr5,148300000,148400000
13350,chr9,63900000,64000000


## Load Data

In [5]:
cell_annot = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/snm3CClustering/ClusteringRecipe/cell_tidy_data.final_annotation.msg'
)
neuron_cell_ids = cell_annot[cell_annot['consensus_cluster_rescued_anno'] == 'NonNeuron'].index


/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
mcds = xr.open_dataset(
    '/home/hanliu/project/mouse_rostral_brain/study/snm3CClustering/ClusteringRecipe/Adata/chrom100k_da_rate.nc'
)

In [7]:
data = mcds['chrom100k_da_rate'].load().sel(cell=neuron_cell_ids, mc_type='CHN', chrom100k=features.index).to_pandas()

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1849: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  3: pd.Panel}


## Predict

In [8]:
predict_proba = clf.predict_proba(data)
predict_proba = pd.DataFrame(predict_proba, columns=clf.classes_, index=data.index)

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=30)]: Done 228 tasks      | elapsed:    0.1s
[Parallel(n_jobs=30)]: Done 500 out of 500 | elapsed:    0.1s finished


In [9]:
predict_proba.to_msgpack('NonNeuronSubType.pred_proba.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [10]:
sub_to_major_map = snmc_meta.set_index('SubType')['MajorType'].to_dict()
sub_to_cc_map = snmc_meta.set_index('SubType')['CellClass'].to_dict()

pred_df = pd.DataFrame(predict_proba.idxmax(axis=1))
pred_df.columns = ['SubType']
pred_df['MajorType'] = pred_df['SubType'].map(sub_to_major_map)
pred_df['CellClass'] = pred_df['SubType'].map(sub_to_cc_map)

In [11]:
pred_df['MajorType'].value_counts()

ODC         716
EC          391
PC          357
MGC         286
VLMC         59
VLMC-Pia     41
OPC          40
ASC          38
ANP          24
Name: MajorType, dtype: int64

In [13]:
pred_df.to_msgpack('NonNeuron.pred.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.
